In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
dfs = pd.DataFrame({})

for i in range(4,10):
    df = pd.read_csv('./trento/sms-call-internet-tn-2013-11-0{}.csv'.format(i), parse_dates=['time'])
    dfs = dfs.append(df)
df = pd.read_csv('./trento/sms-call-internet-tn-2013-11-10.csv', parse_dates=['time'])
dfs = dfs.append(df)
dfs = dfs.fillna(0)
del df

In [3]:
#Group by hours and aggregate values according to each cell/hour
dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-4)
dfgg = dfgr[['hour', 'cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour', 'cellid'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
del dfs
print(dfgg)

#Group by hours and aggregate values according to the hour
#dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
#dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-1)
#dfgg = dfgr[['hour', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
#print(dfgg)

         hour  cellid      smsin     smsout    callin   callout    internet
0           0      38   0.994946   1.038735  0.467851  0.105982   71.180305
1           0      39   1.258125   0.369114  0.141578  0.114836   64.393556
2           0      40   3.173657   1.505841  0.349958  0.060084   72.071691
3           0     154   1.544525   0.565670  0.000000  0.282835   63.078948
4           0     155   2.072674   0.380103  0.124315  0.176232   92.607967
...       ...     ...        ...        ...       ...       ...         ...
1049691   167   11336   0.062597   0.078808  0.263489  0.002686    7.168666
1049692   167   11337   0.365092   1.336275  0.224779  0.002051   17.223766
1049693   167   11452  13.692389  13.022877  0.950413  0.410388  136.094013
1049694   167   11453   3.623349   4.013107  0.294203  0.237093   59.998534
1049695   167   11454   0.112587   1.658289  0.000000  0.001443   21.934601

[1049696 rows x 7 columns]


In [4]:
l=[]
#6134 - 3911
for i in range(3911, 6135, 117):
    for j in range(i, i+20):
        l.append(j)
#print(l)

In [5]:
df20 = pd.DataFrame({})
for el in dfgg.iterrows():
    if(l.count(int(el[1].cellid))==1):
        tmp = pd.DataFrame([[int(el[1].hour), int(el[1].cellid), el[1].smsin, el[1].smsout, el[1].callin, el[1].callout, el[1].internet]], columns=['hour', 'cellid', 'smsin','smsout', 'callin','callout', 'internet'])
        df20=df20.append(tmp)

In [6]:
df20 = df20.set_index(['hour']).sort_index()
print(df20)

      cellid      smsin     smsout    callin   callout    internet
hour                                                              
0       3911  10.329397  11.970914  1.760835  0.658890  132.698970
0       3912  12.824017   8.493987  1.240078  0.724291  147.324070
0       3913   9.362022  14.697714  2.624397  0.438525  185.440675
0       3914   4.668861   7.015766  1.333983  0.640765  152.567759
0       3915   4.790570   3.590049  0.273205  0.234108  183.333266
...      ...        ...        ...       ...       ...         ...
167     6149   4.366485  13.447975  0.778932  1.165898   68.088147
167     6150   4.207260  15.161263  0.239048  0.369234   46.196438
167     6151   1.297493  47.004700  0.392634  0.565737   54.040851
167     6152   0.718600  25.560030  0.180147  0.282804   28.791729
167     6153   0.199112   0.192636  0.032068  0.044937    3.846397

[67200 rows x 6 columns]


In [7]:
cells = df20.cellid
scaler=MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(df20), columns=df20.columns, index=df20.index)

In [8]:
dfgg=scaled
dfgg['cellid']=cells
print(dfgg)

      cellid     smsin    smsout    callin   callout  internet
hour                                                          
0       3911  0.002297  0.003938  0.001136  0.000363  0.005113
0       3912  0.002851  0.002794  0.000800  0.000399  0.005688
0       3913  0.002082  0.004835  0.001694  0.000241  0.007185
0       3914  0.001038  0.002308  0.000861  0.000353  0.005894
0       3915  0.001065  0.001181  0.000176  0.000129  0.007102
...      ...       ...       ...       ...       ...       ...
167     6149  0.000971  0.004424  0.000503  0.000642  0.002575
167     6150  0.000935  0.004988  0.000154  0.000203  0.001715
167     6151  0.000288  0.015464  0.000253  0.000311  0.002023
167     6152  0.000160  0.008409  0.000116  0.000156  0.001031
167     6153  0.000044  0.000063  0.000021  0.000025  0.000051

[67200 rows x 6 columns]


In [9]:
dfgg.to_csv('./classif/20x20/tn/tnwktemp.csv')
print (dfgr.hour.max())

167


In [16]:
dfsi = [] #data from df without indexing
dfsi2 = pd.DataFrame({}) #new df to save
dfso = []
dfso2 = pd.DataFrame({})
dfci = []
dfci2 = pd.DataFrame({})
dfco = []
dfco2 = pd.DataFrame({})
dfin = []
dfin2 = pd.DataFrame({})

dfsi.append(dfgg.loc[0].cellid)
dfso.append(dfgg.loc[0].cellid)
dfci.append(dfgg.loc[0].cellid)
dfco.append(dfgg.loc[0].cellid)
dfin.append(dfgg.loc[0].cellid)

dfsi2["cellid"]=dfsi[0].values
dfso2["cellid"]=dfsi[0].values
dfci2["cellid"]=dfsi[0].values
dfco2["cellid"]=dfsi[0].values
dfin2["cellid"]=dfsi[0].values
for i in range(0, dfgr.hour.max()+1): #dfgr.hour.max()+1
    #row[0] - hour, row[1]: df cols
    dfsi.append(dfgg.loc[i].smsin)
    dfsi2["smsin"+str(i)]=dfsi[i+1].values
    
    dfso.append(dfgg.loc[i].smsout)
    dfso2["smsout"+str(i)]=dfso[i+1].values
    
    dfci.append(dfgg.loc[i].callin)
    dfci2["callin"+str(i)]=dfci[i+1].values
    
    dfco.append(dfgg.loc[i].callout)
    dfco2["callout"+str(i)]=dfco[i+1].values
    
    dfin.append(dfgg.loc[i].internet)
    dfin2["internet"+str(i)]=dfin[i+1].values


In [11]:
dfsi2.to_csv("./classif/20x20/tn/classif20x20week_smsin.csv")
dfsi2.head()

,cellid,smsin0,smsin1,smsin2,smsin3,smsin4,smsin5,smsin6,smsin7,smsin8,...,smsin158,smsin159,smsin160,smsin161,smsin162,smsin163,smsin164,smsin165,smsin166,smsin167
0,3911,0.002297,0.000496,0.000000,0.000630,0.000644,0.000263,0.000616,0.004309,0.006000,...,0.005284,0.005365,0.004742,0.007847,0.005469,0.007014,0.005191,0.004792,0.002741,0.002548
1,3912,0.002851,0.000428,0.000126,0.000640,0.000765,0.000312,0.000828,0.004163,0.006437,...,0.005850,0.006914,0.006103,0.008864,0.006282,0.007612,0.004902,0.005080,0.004819,0.002889
2,3913,0.002082,0.000173,0.000514,0.000061,0.000097,0.000294,0.001401,0.004015,0.010395,...,0.008912,0.007992,0.009707,0.009856,0.008578,0.009144,0.007404,0.006350,0.006930,0.003862
3,3914,0.001038,0.000386,0.000358,0.000125,0.000070,0.000318,0.001069,0.004172,0.008921,...,0.005527,0.006071,0.006780,0.007570,0.007219,0.007024,0.005731,0.004689,0.004208,0.002358
4,3915,0.001065,0.000308,0.000320,0.000012,0.000193,0.001528,0.000463,0.005440,0.011199,...,0.003955,0.005279,0.003570,0.005534,0.006073,0.005212,0.005047,0.004502,0.002931,0.002060


In [12]:
dfso2.to_csv("./classif/20x20/tn/classif20x20week_smsout.csv")
dfso2.head()

,cellid,smsout0,smsout1,smsout2,smsout3,smsout4,smsout5,smsout6,smsout7,smsout8,...,smsout158,smsout159,smsout160,smsout161,smsout162,smsout163,smsout164,smsout165,smsout166,smsout167
0,3911,0.003938,0.002909,0.002363,0.002284,0.002233,0.001676,0.001025,0.002610,0.004961,...,0.005359,0.005246,0.003767,0.005409,0.006920,0.010114,0.006461,0.009503,0.005857,0.003236
1,3912,0.002794,0.000791,0.000275,0.000000,0.000000,0.000232,0.001140,0.002200,0.003128,...,0.005360,0.005768,0.006254,0.006681,0.006522,0.008766,0.004922,0.006501,0.004738,0.002619
2,3913,0.004835,0.001541,0.001285,0.001122,0.001116,0.000731,0.001176,0.003794,0.007780,...,0.007956,0.006835,0.008850,0.011102,0.009417,0.009626,0.009863,0.007194,0.009746,0.005382
3,3914,0.002308,0.000320,0.000583,0.000493,0.000608,0.000237,0.002078,0.005019,0.005881,...,0.006145,0.004418,0.006035,0.007151,0.006814,0.006886,0.006704,0.006506,0.005783,0.003097
4,3915,0.001181,0.000144,0.000146,0.000133,0.000121,0.000088,0.001296,0.005669,0.005483,...,0.003601,0.003113,0.002979,0.005477,0.004235,0.004713,0.004093,0.005291,0.003524,0.001778


In [13]:
dfci2.to_csv("./classif/20x20/tn/classif20x20week_callin.csv")
dfci2.head()

,cellid,callin0,callin1,callin2,callin3,callin4,callin5,callin6,callin7,callin8,...,callin158,callin159,callin160,callin161,callin162,callin163,callin164,callin165,callin166,callin167
0,3911,0.001136,0.000327,0.000156,0.000142,0.000000,0.000284,0.001113,0.005424,0.014285,...,0.005531,0.005616,0.007276,0.010946,0.006414,0.005776,0.007277,0.003904,0.001454,0.000568
1,3912,0.000800,0.000177,0.000000,0.000179,0.000024,0.000159,0.000444,0.004503,0.011812,...,0.005995,0.005667,0.006929,0.011179,0.005980,0.005543,0.006977,0.003524,0.000897,0.000419
2,3913,0.001694,0.000524,0.000253,0.000270,0.000216,0.000450,0.000960,0.006404,0.019380,...,0.008526,0.006548,0.009244,0.014207,0.008215,0.007107,0.008503,0.004384,0.001858,0.001204
3,3914,0.000861,0.000744,0.000339,0.000310,0.000371,0.000736,0.001306,0.006458,0.013506,...,0.006528,0.006389,0.009092,0.011494,0.009290,0.008513,0.006739,0.004429,0.002189,0.001014
4,3915,0.000176,0.000416,0.000834,0.000022,0.000151,0.000191,0.000609,0.005599,0.013053,...,0.005584,0.004326,0.006044,0.006762,0.008261,0.007664,0.005668,0.003539,0.002664,0.000998


In [14]:
dfco2.to_csv("./classif/20x20/tn/classif20x20week_callout.csv")
dfco2.head()

,cellid,callout0,callout1,callout2,callout3,callout4,callout5,callout6,callout7,callout8,...,callout158,callout159,callout160,callout161,callout162,callout163,callout164,callout165,callout166,callout167
0,3911,0.000363,0.000266,0.000109,0.000156,0.000109,0.000024,0.000816,0.006975,0.015811,...,0.005966,0.007628,0.007544,0.011838,0.010716,0.010989,0.007561,0.004366,0.002248,0.000899
1,3912,0.000399,0.000265,0.000000,0.000004,0.000000,0.000000,0.000955,0.006204,0.011442,...,0.004782,0.005978,0.006093,0.009519,0.008728,0.008475,0.006822,0.004536,0.001701,0.000726
2,3913,0.000241,0.000015,0.000054,0.000101,0.000054,0.000024,0.002713,0.008443,0.016158,...,0.009597,0.007359,0.009447,0.012694,0.010386,0.011608,0.007014,0.004715,0.002269,0.001625
3,3914,0.000353,0.000101,0.000067,0.000075,0.000047,0.000608,0.003460,0.008061,0.014728,...,0.008688,0.008905,0.009428,0.010868,0.010906,0.009956,0.006311,0.003204,0.001817,0.001221
4,3915,0.000129,0.000383,0.000505,0.000294,0.000012,0.000357,0.005847,0.005772,0.014130,...,0.006280,0.008092,0.006592,0.007093,0.006524,0.007829,0.004600,0.003217,0.001592,0.002028


In [15]:
dfin2.to_csv("./classif/20x20/tn/classif20x20week_internet.csv")
dfin2.head()

,cellid,internet0,internet1,internet2,internet3,internet4,internet5,internet6,internet7,internet8,...,internet158,internet159,internet160,internet161,internet162,internet163,internet164,internet165,internet166,internet167
0,3911,0.005113,0.004020,0.003597,0.003422,0.003535,0.003707,0.005363,0.010151,0.011197,...,0.009580,0.008619,0.010331,0.009969,0.009127,0.007455,0.007207,0.006631,0.005753,0.006155
1,3912,0.005688,0.004450,0.003734,0.003947,0.004340,0.004474,0.005526,0.009378,0.010903,...,0.009970,0.009089,0.011769,0.010814,0.010571,0.009155,0.009535,0.007960,0.007208,0.007750
2,3913,0.007185,0.006068,0.005257,0.005804,0.006768,0.007093,0.008546,0.012286,0.015600,...,0.010986,0.012401,0.015108,0.012760,0.013982,0.013405,0.013727,0.011220,0.010121,0.010243
3,3914,0.005894,0.005187,0.004381,0.004540,0.005093,0.005453,0.007449,0.009809,0.011473,...,0.009049,0.010214,0.011830,0.010420,0.011842,0.010563,0.011076,0.010551,0.008650,0.007422
4,3915,0.007102,0.006717,0.005705,0.005369,0.005515,0.005677,0.006684,0.008094,0.013742,...,0.007591,0.008628,0.010370,0.009180,0.009914,0.008962,0.010563,0.011591,0.009156,0.008654
